In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt

#### 结合了高斯分布

#### GELU 激活函数

公式：

$$
\text{GELU}(x) = x \cdot \Phi(x)
$$

其中，$\Phi(x)$ 是标准正态分布的累积分布函数，定义为：

$$
\Phi(x) = \frac{1}{2} \left( 1 + \text{erf}\left( \frac{x}{\sqrt{2}} \right) \right)
$$


`erf` 是 **误差函数（Error Function）**，定义为：

$$
\text{erf}(x) = \frac{2}{\sqrt{\pi}} \int_0^x e^{-t^2} \, dt
$$

### GELU 激活函数近似公式（tanh 近似）

$$
\text{GELU}(x) \approx 0.5 \, x \left( 1 + \tanh\left[ \sqrt{\frac{2}{\pi}} \left( x + 0.044715 \, x^3 \right) \right] \right)
$$

解释：
- 使用 \(\tanh\) 对标准 GELU 的正态累积分布函数进行近似。
- 提高计算效率，同时保留平滑非线性特性。
- 常用于 Transformer 和大模型的实际实现中。


In [2]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x): 
        return 0.5 * x * (1 + torch.tanh( torch.sqrt(torch.tensor(2.0 / torch.pi)) *  (x + 0.044715 * torch.pow(x, 3))))

In [5]:
GPT_CONFIG_124M = { 
 "vocab_size": 50257, # 词汇表大小
 "context_length": 1024, # 上下文长度
 "emb_dim": 768, # 嵌入维度
 "n_heads": 12, # 注意力头的数量
 "n_layers": 12, # 层数
 "drop_rate": 0.1, # dropout 率
 "qkv_bias": False # 查询-键-值偏置
}

### 比较GLEU和ReLU

In [3]:
class FeedForward(nn.Module): 
 def __init__(self, cfg): 
    super().__init__() 
    self.layers = nn.Sequential( 
        nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), 
        GELU(), 
        nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]), 
    ) 
 def forward(self, x): 
    return self.layers(x)

In [6]:
ffn = FeedForward(GPT_CONFIG_124M) 
x = torch.rand(2, 3, 768) 
out = ffn(x) 
print(out.shape)

torch.Size([2, 3, 768])
